# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# Extract data

## Read data
Define place of files.

In [2]:
data_folder = "data"
features_path = f"{data_folder}/tabular_data.csv"
hashed_features_path = f"{data_folder}/hashed_feature.csv"
labels_path = f"{data_folder}/train.csv"
test_df_path = f"{data_folder}/test.csv"

Features.

In [3]:

features = pd.read_csv(features_path,index_col=0)
features.head()

,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,110.0,55.0,0.432017,0.000000,176.78,0.0,0.323712,NaN,147.0,0.0,176.78,55.0,0.248711,0.001373,0.0,0.0,0.0,0.476549,333.60,0.277617,55.0,0.0,55.0,21.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,1.00,55.000000,93.0,5562.0,93.0,0.040283,182.0,0.000000,43.50,0.0,0.0,0.00,0.00,0,0.000000,0.0,55.0,2.0,0.526552,145.0,133.28,0.0
0,2,110.0,110.0,0.397517,0.000000,315.42,0.0,0.316798,NaN,135.0,0.0,315.42,110.0,0.261744,-0.002248,0.0,0.0,0.0,0.210951,428.78,0.290904,110.0,0.0,110.0,26.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.77,110.000000,114.0,5532.0,114.0,0.047588,181.0,0.000000,85.45,0.0,0.0,15.75,0.00,0,0.000000,0.0,110.0,1.0,0.481063,130.0,229.97,0.0
0,3,110.0,55.0,0.359440,0.000000,354.55,0.0,0.339188,NaN,181.0,0.0,354.48,55.0,0.302164,0.002083,0.0,0.0,0.0,0.058675,526.38,0.284842,55.0,0.0,55.0,36.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.05,85249.281058,135.0,5501.0,136.0,0.051470,180.0,0.000000,122.70,0.0,0.0,0.00,0.07,0,0.000000,0.0,55.0,1.0,0.509598,180.0,231.78,0.0
0,4,110.0,55.0,0.285707,0.000000,229.98,0.0,0.415428,NaN,142.0,0.0,229.98,55.0,0.347660,0.005207,0.0,0.0,0.0,0.966854,398.98,0.285235,55.0,0.0,55.0,30.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.00,55.000000,102.0,5471.0,102.0,0.076190,179.0,0.000000,46.15,0.0,0.0,0.00,0.00,0,0.000000,0.0,55.0,0.0,0.680089,142.0,183.83,0.0
0,5,110.0,55.0,0.101487,444.730391,307.12,0.0,0.569670,NaN,92.0,0.0,306.17,55.0,0.438425,0.025736,0.0,21.0,0.0,0.700180,215.17,0.161147,55.0,0.0,55.0,20.0,0.0,11e623a37e87cf7995c466723ec99688d55cae8c,NaN,0.0,0.00,55.000000,82.0,5440.0,95.0,0.273205,178.0,373.498562,150.33,0.0,0.0,26.75,0.95,0,20.014485,0.0,55.0,0.0,0.776175,85.0,155.83,0.0


Hashed data about user.

In [4]:

hashed_features = pd.read_csv(hashed_features_path, index_col=0)
hashed_features.head()

,feature_50
id,
0,7984b0a0e139cabadb5afc7756d473fb34d23819
0,47904b3331202e9881d003ff449c2eabfbc75460
0,a99c4b3af723874ddd85af322beea81b64437294
0,4dce5381031a88aed6b12ef71b6f7c3148e7b3c8
0,d62db721202cb6636887f450a7b77fa97db03b05


In [5]:

labels = pd.read_csv(labels_path, index_col=0)
labels.head()

,target
id,
0,0
1,0
2,1
3,0
4,1


In [6]:

test = pd.read_csv(test_df_path, index_col=0)
test.head()


,score
id,
4084,NaN
4085,NaN
4086,NaN
4087,NaN
4088,NaN


## Build data frames
First build train data frame

In [7]:

train_users_id = labels.index.unique()
train_users = features.loc[train_users_id]
train_df = pd.concat((train_users, labels), axis=1)
train_df.head()

,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,110.0,55.0,0.432017,0.000000,176.78,0.0,0.323712,NaN,147.0,0.0,176.78,55.0,0.248711,0.001373,0.0,0.0,0.0,0.476549,333.60,0.277617,55.0,0.0,55.0,21.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,1.00,55.000000,93.0,5562.0,93.0,0.040283,182.0,0.000000,43.50,0.0,0.0,0.00,0.00,0,0.000000,0.0,55.0,2.0,0.526552,145.0,133.28,0.0,0
0,2,110.0,110.0,0.397517,0.000000,315.42,0.0,0.316798,NaN,135.0,0.0,315.42,110.0,0.261744,-0.002248,0.0,0.0,0.0,0.210951,428.78,0.290904,110.0,0.0,110.0,26.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.77,110.000000,114.0,5532.0,114.0,0.047588,181.0,0.000000,85.45,0.0,0.0,15.75,0.00,0,0.000000,0.0,110.0,1.0,0.481063,130.0,229.97,0.0,0
0,3,110.0,55.0,0.359440,0.000000,354.55,0.0,0.339188,NaN,181.0,0.0,354.48,55.0,0.302164,0.002083,0.0,0.0,0.0,0.058675,526.38,0.284842,55.0,0.0,55.0,36.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.05,85249.281058,135.0,5501.0,136.0,0.051470,180.0,0.000000,122.70,0.0,0.0,0.00,0.07,0,0.000000,0.0,55.0,1.0,0.509598,180.0,231.78,0.0,0
0,4,110.0,55.0,0.285707,0.000000,229.98,0.0,0.415428,NaN,142.0,0.0,229.98,55.0,0.347660,0.005207,0.0,0.0,0.0,0.966854,398.98,0.285235,55.0,0.0,55.0,30.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.00,55.000000,102.0,5471.0,102.0,0.076190,179.0,0.000000,46.15,0.0,0.0,0.00,0.00,0,0.000000,0.0,55.0,0.0,0.680089,142.0,183.83,0.0,0
0,5,110.0,55.0,0.101487,444.730391,307.12,0.0,0.569670,NaN,92.0,0.0,306.17,55.0,0.438425,0.025736,0.0,21.0,0.0,0.700180,215.17,0.161147,55.0,0.0,55.0,20.0,0.0,11e623a37e87cf7995c466723ec99688d55cae8c,NaN,0.0,0.00,55.000000,82.0,5440.0,95.0,0.273205,178.0,373.498562,150.33,0.0,0.0,26.75,0.95,0,20.014485,0.0,55.0,0.0,0.776175,85.0,155.83,0.0,0


Second get test data frame

In [8]:
test_users_id = test.index.unique()
test_df = features.loc[test_users_id]
test_df.head()

,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4084,1,110.0,77.66,0.494997,0.000000,209.00,0.0,0.201109,141.82,35.0,3.27,183.28,77.66,0.181976,0.002610,0.0,0.0,0.000000,0.838393,174.50,NaN,75.00,2.66,77.66,9.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,20.0,3.27,32.58,60.0,29.0,4451.0,44.0,0.032862,146.0,0.000000,62.53,0.0,0.0,0.10,0.87,0,-0.000000,0.0,75.00,3.0,0.035053,31.0,120.75,2.0
4084,2,110.0,161.86,0.448919,0.000000,193.65,0.0,0.167043,114.73,36.0,1.30,160.38,161.86,0.207897,0.004074,0.0,0.0,16181.098369,0.209477,154.73,NaN,150.00,11.86,161.86,-8.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,24.0,0.00,33.35,165.0,32.0,4421.0,47.0,0.045954,145.0,0.000000,43.25,0.0,0.0,6.65,1.69,0,-0.000000,0.0,150.00,2.0,0.037252,29.0,117.13,1.0
4084,3,110.0,85.00,0.385688,0.000000,130.27,0.0,0.161869,263.30,152.0,5.30,122.10,85.00,0.255406,0.005041,1.0,0.0,1.000000,0.626729,275.05,NaN,80.00,5.00,85.00,11.0,1.0,f9ae10397ed62e79c644d06842bbcd92963a5927,40.0,0.00,11.75,100.0,51.0,4390.0,61.0,0.062920,144.0,0.000000,28.30,0.0,0.0,0.00,2.87,0,0.000000,1.0,80.00,1.0,0.013951,151.0,93.80,2.0
4084,4,110.0,78.00,0.365817,0.000000,157.03,0.0,0.156286,787.50,215.0,1.38,148.10,78.00,0.280130,0.005876,0.0,0.0,0.000000,0.618390,804.75,NaN,75.00,3.00,78.00,3.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,27.0,0.00,17.25,85.0,30.0,4360.0,57.0,0.075233,143.0,0.000000,5.63,0.0,0.0,0.00,7.55,0,0.000000,0.0,75.00,1.0,0.005658,214.0,142.47,1.0
4084,5,110.0,85.08,0.105262,2041.945145,67.42,0.0,0.113863,430.43,126.0,10.72,51.70,85.08,0.355205,0.042197,0.0,8.0,1.000000,0.260960,459.53,NaN,75.08,10.00,85.08,5.0,1.0,f9ae10397ed62e79c644d06842bbcd92963a5927,32.0,0.00,29.10,85.0,37.0,4329.0,62.0,0.359863,142.0,2040.800154,3.73,0.0,0.0,0.00,5.00,0,0.056302,1.0,75.08,5.0,0.006493,121.0,47.97,3.0


# Preprocessing
Check type of each column. As we can see 'feature_25' is object. All other 

In [9]:
features.dtypes

period          int64
feature_0     float64
feature_1     float64
feature_2     float64
feature_3     float64
feature_4     float64
feature_5     float64
feature_6     float64
feature_7     float64
feature_8     float64
feature_9     float64
feature_10    float64
feature_11    float64
feature_12    float64
feature_13    float64
feature_14    float64
feature_15    float64
feature_16    float64
feature_17    float64
feature_18    float64
feature_19    float64
feature_20    float64
feature_21    float64
feature_22    float64
feature_23    float64
feature_24    float64
feature_25     object
feature_26    float64
feature_27    float64
feature_28    float64
feature_29    float64
feature_30    float64
feature_31    float64
feature_32    float64
feature_33    float64
feature_34    float64
feature_35    float64
feature_36    float64
feature_37    float64
feature_38    float64
feature_39    float64
feature_40    float64
feature_41      int64
feature_42    float64
feature_43    float64
feature_44

Let`s check does 'feature_25' has missed values. 

In [10]:
features["feature_25"].isnull().any()

False

Index is not unique now in train data frame. We should change it.

In [11]:
train_df = train_df.reset_index()
train_df.head(2)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,1,110.0,55.0,0.432017,0.0,176.78,0.0,0.323712,NaN,147.0,0.0,176.78,55.0,0.248711,0.001373,0.0,0.0,0.0,0.476549,333.60,0.277617,55.0,0.0,55.0,21.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,1.00,55.0,93.0,5562.0,93.0,0.040283,182.0,0.0,43.50,0.0,0.0,0.00,0.0,0,0.0,0.0,55.0,2.0,0.526552,145.0,133.28,0.0,0
1,0,2,110.0,110.0,0.397517,0.0,315.42,0.0,0.316798,NaN,135.0,0.0,315.42,110.0,0.261744,-0.002248,0.0,0.0,0.0,0.210951,428.78,0.290904,110.0,0.0,110.0,26.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.77,110.0,114.0,5532.0,114.0,0.047588,181.0,0.0,85.45,0.0,0.0,15.75,0.0,0,0.0,0.0,110.0,1.0,0.481063,130.0,229.97,0.0,0


Index is not unique now in test data so we need to change it.

In [12]:
test_df = test_df.reset_index()
test_df.head(2)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,4084,1,110.0,77.66,0.494997,0.0,209.00,0.0,0.201109,141.82,35.0,3.27,183.28,77.66,0.181976,0.002610,0.0,0.0,0.000000,0.838393,174.50,NaN,75.0,2.66,77.66,9.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,20.0,3.27,32.58,60.0,29.0,4451.0,44.0,0.032862,146.0,0.0,62.53,0.0,0.0,0.10,0.87,0,-0.0,0.0,75.0,3.0,0.035053,31.0,120.75,2.0
1,4084,2,110.0,161.86,0.448919,0.0,193.65,0.0,0.167043,114.73,36.0,1.30,160.38,161.86,0.207897,0.004074,0.0,0.0,16181.098369,0.209477,154.73,NaN,150.0,11.86,161.86,-8.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,24.0,0.00,33.35,165.0,32.0,4421.0,47.0,0.045954,145.0,0.0,43.25,0.0,0.0,6.65,1.69,0,-0.0,0.0,150.0,2.0,0.037252,29.0,117.13,1.0


## Remove outliers

Discover numerical features

In [13]:
not_numerical_features = ["period", "feature_25", "id"]
numerical_features = [feature for feature in features.columns if feature not in not_numerical_features]
train_num_df = train_df[numerical_features]
train_num_df = train_num_df.fillna(train_num_df.mean())
train_num_df.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,110.0,55.0,0.432017,0.0,176.78,0.0,0.323712,390.162546,147.0,0.0,176.78,55.0,0.248711,0.001373,0.0,0.0,0.0,0.476549,333.60,0.277617,55.0,0.0,55.0,21.0,0.0,100.821298,0.0,1.00,55.000000,93.0,5562.0,93.0,0.040283,182.0,0.0,43.50,0.0,0.0,0.00,0.00,0,0.0,0.0,55.0,2.0,0.526552,145.0,133.28,0.0
1,110.0,110.0,0.397517,0.0,315.42,0.0,0.316798,390.162546,135.0,0.0,315.42,110.0,0.261744,-0.002248,0.0,0.0,0.0,0.210951,428.78,0.290904,110.0,0.0,110.0,26.0,0.0,100.821298,0.0,0.77,110.000000,114.0,5532.0,114.0,0.047588,181.0,0.0,85.45,0.0,0.0,15.75,0.00,0,0.0,0.0,110.0,1.0,0.481063,130.0,229.97,0.0
2,110.0,55.0,0.359440,0.0,354.55,0.0,0.339188,390.162546,181.0,0.0,354.48,55.0,0.302164,0.002083,0.0,0.0,0.0,0.058675,526.38,0.284842,55.0,0.0,55.0,36.0,0.0,100.821298,0.0,0.05,85249.281058,135.0,5501.0,136.0,0.051470,180.0,0.0,122.70,0.0,0.0,0.00,0.07,0,0.0,0.0,55.0,1.0,0.509598,180.0,231.78,0.0


In [14]:
res = np.abs(train_num_df - train_num_df.mean()) <= (3 * train_num_df.std())
outliers_index = res.all(axis=1).values
outliers_index

array([ True,  True, False, ..., False, False, False])

Get from train_df only not outliers.

In [15]:
train_df = train_df[outliers_index]
train_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,1,110.0,55.0,0.432017,0.0,176.78,0.0,0.323712,NaN,147.0,0.0,176.78,55.0,0.248711,0.001373,0.0,0.0,0.0,0.476549,333.60,0.277617,55.0,0.0,55.0,21.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,1.00,55.0,93.0,5562.0,93.0,0.040283,182.0,0.0,43.50,0.0,0.0,0.00,0.0,0,0.0,0.0,55.0,2.0,0.526552,145.0,133.28,0.0,0
1,0,2,110.0,110.0,0.397517,0.0,315.42,0.0,0.316798,NaN,135.0,0.0,315.42,110.0,0.261744,-0.002248,0.0,0.0,0.0,0.210951,428.78,0.290904,110.0,0.0,110.0,26.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.77,110.0,114.0,5532.0,114.0,0.047588,181.0,0.0,85.45,0.0,0.0,15.75,0.0,0,0.0,0.0,110.0,1.0,0.481063,130.0,229.97,0.0,0
3,0,4,110.0,55.0,0.285707,0.0,229.98,0.0,0.415428,NaN,142.0,0.0,229.98,55.0,0.347660,0.005207,0.0,0.0,0.0,0.966854,398.98,0.285235,55.0,0.0,55.0,30.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,NaN,0.0,0.00,55.0,102.0,5471.0,102.0,0.076190,179.0,0.0,46.15,0.0,0.0,0.00,0.0,0,0.0,0.0,55.0,0.0,0.680089,142.0,183.83,0.0,0


## Replace missing data

### Prepare replace data
For train tepends on target and period:

In [16]:
train_mean_values = train_df.groupby(["target", "period"]).mean().fillna(0)
train_mean_values.head(3)

id   feature_0  ...  feature_48  feature_49
target period                           ...                        
0      1       2011.210133  110.025599  ...  218.254391    6.866558
       2       2039.576754  110.011192  ...  232.956051    7.253773
       3       2026.034067  110.012945  ...  222.154943    7.266234

[3 rows x 50 columns]

For test depends on period:

In [17]:
test_mean_values = train_df.groupby("period").mean().fillna(0)
test_mean_values.head(3)

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
period,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2031.356602,110.025173,74.601172,0.221196,1455.982648,306.168310,674.258301,0.446263,321.860734,99.903291,12.692292,292.495645,65.071688,0.285232,0.040359,0.789837,9.983713,1.011047,0.503529,340.507535,0.247406,65.901296,7.233895,74.311301,22.039722,0.759056,73.870093,2.848444,14.692573,71.082253,97.999594,3041.886493,99.669792,0.203176,94.466775,727.375166,76.059910,1.189311,0.004152,1.173252,1.421940,0.0,335.632088,0.490212,65.869502,8.507111,0.439904,99.369618,216.045960,7.106418,0.252094
2,2039.925176,110.009688,76.701105,0.216872,1518.790844,309.182532,740.091098,0.456562,341.772366,110.368354,12.904966,293.858651,68.786255,0.287461,0.040851,0.821368,10.729272,1.044624,0.493908,358.851890,0.245725,67.215290,7.776013,76.734182,20.452654,0.771115,83.438420,2.602626,14.540672,72.708525,105.778806,2986.247475,107.441998,0.206836,93.500211,702.749909,66.779185,1.642438,0.004896,1.279983,1.394548,0.0,263.000076,0.487516,67.085813,9.037505,0.447324,108.812211,228.301479,7.661901,0.245969
3,2033.960207,110.012689,75.430730,0.211764,1564.237676,311.625197,706.196818,0.460238,322.625004,103.065574,13.339576,296.239098,66.828454,0.286509,0.041607,0.955665,9.769357,1.223093,0.505043,339.820814,0.245305,65.433150,8.255747,75.106708,23.811313,0.893547,78.364230,2.981522,14.378671,71.739760,103.969660,2981.700327,104.794988,0.210493,94.596880,792.036939,79.331553,1.491839,0.004871,1.089399,1.440201,0.0,276.472606,0.543737,65.226499,8.709188,0.444984,103.807519,218.020505,7.638900,0.240748


### Train data


Replace missing values in train data frame.

In [18]:
def replace_train_missing(raw):
  period = raw["period"]
  target = raw["target"]
  indices_where_nan = raw[raw.isna()].index
  raw[indices_where_nan] = train_mean_values.loc[(target, period), indices_where_nan]
  return raw

train_df = train_df.apply(replace_train_missing, axis=1)
train_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,1,110.0,55.0,0.432017,0.0,176.78,0.0,0.323712,336.901242,147.0,0.0,176.78,55.0,0.248711,0.001373,0.0,0.0,0.0,0.476549,333.60,0.277617,55.0,0.0,55.0,21.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,70.167658,0.0,1.00,55.0,93.0,5562.0,93.0,0.040283,182.0,0.0,43.50,0.0,0.0,0.00,0.0,0,0.0,0.0,55.0,2.0,0.526552,145.0,133.28,0.0,0
1,0,2,110.0,110.0,0.397517,0.0,315.42,0.0,0.316798,360.228744,135.0,0.0,315.42,110.0,0.261744,-0.002248,0.0,0.0,0.0,0.210951,428.78,0.290904,110.0,0.0,110.0,26.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,79.787068,0.0,0.77,110.0,114.0,5532.0,114.0,0.047588,181.0,0.0,85.45,0.0,0.0,15.75,0.0,0,0.0,0.0,110.0,1.0,0.481063,130.0,229.97,0.0,0
3,0,4,110.0,55.0,0.285707,0.0,229.98,0.0,0.415428,343.926172,142.0,0.0,229.98,55.0,0.347660,0.005207,0.0,0.0,0.0,0.966854,398.98,0.285235,55.0,0.0,55.0,30.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,76.918712,0.0,0.00,55.0,102.0,5471.0,102.0,0.076190,179.0,0.0,46.15,0.0,0.0,0.00,0.0,0,0.0,0.0,55.0,0.0,0.680089,142.0,183.83,0.0,0


Assert all nan values are replaced in train data.

In [19]:
train_df.isna().any().any()

False

### Test data

Replace missing values in test data frame.

In [20]:
def replace_test_missing(raw):
  period = raw["period"]
  indices_where_nan = raw[raw.isna()].index
  raw[indices_where_nan] = test_mean_values.loc[period, indices_where_nan]
  return raw

test_df = test_df.apply(replace_test_missing, axis=1)
test_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,4084,1,110.0,77.66,0.494997,0.0,209.00,0.0,0.201109,141.82,35.0,3.27,183.28,77.66,0.181976,0.002610,0.0,0.0,0.000000,0.838393,174.50,0.247406,75.0,2.66,77.66,9.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,20.0,3.27,32.58,60.0,29.0,4451.0,44.0,0.032862,146.0,0.0,62.53,0.0,0.0,0.10,0.87,0,-0.0,0.0,75.0,3.0,0.035053,31.0,120.75,2.0
1,4084,2,110.0,161.86,0.448919,0.0,193.65,0.0,0.167043,114.73,36.0,1.30,160.38,161.86,0.207897,0.004074,0.0,0.0,16181.098369,0.209477,154.73,0.245725,150.0,11.86,161.86,-8.0,0.0,f9ae10397ed62e79c644d06842bbcd92963a5927,24.0,0.00,33.35,165.0,32.0,4421.0,47.0,0.045954,145.0,0.0,43.25,0.0,0.0,6.65,1.69,0,-0.0,0.0,150.0,2.0,0.037252,29.0,117.13,1.0
2,4084,3,110.0,85.00,0.385688,0.0,130.27,0.0,0.161869,263.30,152.0,5.30,122.10,85.00,0.255406,0.005041,1.0,0.0,1.000000,0.626729,275.05,0.245305,80.0,5.00,85.00,11.0,1.0,f9ae10397ed62e79c644d06842bbcd92963a5927,40.0,0.00,11.75,100.0,51.0,4390.0,61.0,0.062920,144.0,0.0,28.30,0.0,0.0,0.00,2.87,0,0.0,1.0,80.0,1.0,0.013951,151.0,93.80,2.0


Assert there is no nan values now in test data.

In [21]:
test_df.isna().any().any()

False

#Processing

In [22]:
num_scaler = StandardScaler().fit(train_df[numerical_features])
hash_scaler = LabelBinarizer().fit(train_df["feature_25"])

In [23]:
def normalize(data):
  data = data.copy()
  num_data = data[numerical_features]
  string_data = data["feature_25"]
  normed_num_data = num_scaler.transform(num_data)
  normed_string_data = hash_scaler.transform(string_data)
  data[numerical_features] = normed_num_data
  del data["feature_25"]
  cols = ["feature_" + col_name for col_name in hash_scaler.classes_]
  data[cols] = normed_string_data
  return data

In [24]:
train_df = normalize(train_df)
train_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target,feature_11e623a37e87cf7995c466723ec99688d55cae8c,feature_390c481a740bb1c12c57f33dc5263ced2ab11796,feature_aa2c96dacf00c451ef465f6115a45a20bccf1256,feature_b204e1e3743cc314841070435211e25f4602b9fd,feature_cf3cca1da7361bd988642600e76c4a3021be8ccf,feature_cf7413906c99be47067cb3fb6299959857710d77,feature_d458ece2abeae803254c5d442b2e2a80f58a4153,feature_f9ae10397ed62e79c644d06842bbcd92963a5927
0,0,1,-0.145749,-0.321169,1.284885,-0.423812,-0.430703,-0.248678,-0.550014,0.023489,0.431641,-0.556208,-0.388646,-0.138730,-0.318938,-0.700217,-0.258374,-0.745992,-0.268389,-0.080761,-0.043934,0.366718,-0.154123,-0.431552,-0.317875,-0.064902,-0.250632,-0.125785,-0.453233,-0.550187,-0.193706,-0.117584,1.594707,-0.114261,-1.171955,1.504978,-0.332461,-0.370500,-0.196656,-0.126987,-0.187574,-0.392605,0.0,-0.23158,-0.419912,-0.152786,-0.559956,0.335922,0.474327,-0.377473,-0.566729,0,0,0,0,0,0,0,0,1
1,0,2,-0.145749,0.750577,1.077670,-0.423812,-0.009643,-0.248678,-0.577120,0.100046,0.311785,-0.556208,0.040165,0.791198,-0.189772,-0.761141,-0.258374,-0.745992,-0.268389,-0.998543,0.264132,0.495786,1.048612,-0.431552,0.751855,0.128898,-0.250632,0.002317,-0.453233,-0.559887,0.601792,0.101579,1.576380,0.073958,-1.122802,1.488117,-0.332461,0.087098,-0.196656,-0.126987,2.254323,-0.392605,0.0,-0.23158,-0.419912,1.050944,-0.645213,0.209318,0.299758,0.018168,-0.566729,0,0,0,0,0,0,0,0,1
3,0,4,-0.145749,-0.321169,0.406117,-0.423812,-0.269130,-0.248678,-0.190452,0.046544,0.381701,-0.556208,-0.224100,-0.138730,0.661692,-0.635703,-0.258374,-0.745992,-0.268389,1.613501,0.167679,0.440720,-0.154123,-0.431552,-0.317875,0.283938,-0.250632,-0.035881,-0.453233,-0.592358,-0.193706,-0.023657,1.539114,-0.033596,-0.930340,1.454394,-0.332461,-0.341593,-0.196656,-0.126987,-0.187574,-0.392605,0.0,-0.23158,-0.419912,-0.152786,-0.730471,0.763245,0.439413,-0.170630,-0.566729,0,0,0,0,0,0,0,0,1


## Choosing model by brute force

Function for choosing models

In [25]:
informative_features = [feature_name for feature_name in train_df.columns if feature_name.startswith("feature")]
informative_features

['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_11e623a37e87cf7995c466723ec99688d55cae8c',
 'feature_390c481a740bb1c12c57f33dc5263ced2ab11796',
 'feature_aa2c96dacf00c451ef465f6115a45a20bccf1256',
 'feature_b204e1e3743cc314841070435211e25f4602b9fd',
 'feature_cf3cca1da7361bd988642600e76c4a3021be8ccf',
 'feature_

In [26]:
classifiers = [
             RandomForestClassifier(),
             SVC(kernel="linear", C=0.02),
             SVC(gamma=2, C=1),
             QuadraticDiscriminantAnalysis()
]

In [27]:
def choose_best_model():
  results = list()
  for classifier in classifiers:
    for i in range(5):
      x_train, x_test, y_train, y_test = train_test_split(train_df[informative_features], train_df["target"], test_size=0.1, random_state=i)
      model = classifier.fit(x_train, y_train)
      y_predicted = model.predict(x_test)
      f1_calc_score = f1_score(y_test, y_predicted)
      results.append((model, f1_calc_score))
  return max(results, key=lambda pair: pair[1])

In [28]:
best_model, best_score = choose_best_model()
best_score

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.5921397379912664

In [30]:
normalized_test_data = normalize(test_df)
normalized_test_data.head()

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_11e623a37e87cf7995c466723ec99688d55cae8c,feature_390c481a740bb1c12c57f33dc5263ced2ab11796,feature_aa2c96dacf00c451ef465f6115a45a20bccf1256,feature_b204e1e3743cc314841070435211e25f4602b9fd,feature_cf3cca1da7361bd988642600e76c4a3021be8ccf,feature_cf7413906c99be47067cb3fb6299959857710d77,feature_d458ece2abeae803254c5d442b2e2a80f58a4153,feature_f9ae10397ed62e79c644d06842bbcd92963a5927
0,4084,1,-0.145749,0.120390,1.663155,-0.423812,-0.332848,-0.248678,-1.030664,-0.616735,-0.687014,-0.413489,-0.368542,0.244401,-0.980305,-0.679405,-0.258374,-0.745992,-0.268389,1.169599,-0.558888,0.073242,0.283235,-0.282858,0.122854,-0.530021,-0.250632,-0.793870,0.077854,0.781583,-0.121388,-0.785509,0.915981,-0.553439,-1.221886,0.897972,-0.332461,-0.162917,-0.196656,-0.126987,-0.172070,-0.179944,0.0,-0.231580,-0.419912,0.284934,-0.474699,-1.032009,-0.852394,-0.428744,-0.409401,0,0,0,0,0,0,0,1
1,4084,2,-0.145749,1.761135,1.386403,-0.423812,-0.379467,-0.248678,-1.164212,-0.705640,-0.677026,-0.499470,-0.439371,1.668036,-0.723416,-0.654774,-0.258374,-0.745992,3819.106371,-1.003637,-0.622877,0.056918,1.923328,0.231422,1.760513,-1.188941,-0.250632,-0.740602,-0.453233,0.814055,1.397290,-0.754200,0.897654,-0.526550,-1.133792,0.881111,-0.332461,-0.373227,-0.196656,-0.126987,0.843449,0.020495,0.0,-0.231580,-0.419912,1.926385,-0.559956,-1.025889,-0.875670,-0.443557,-0.488065,0,0,0,0,0,0,0,1
2,4084,3,-0.145749,0.263420,1.006624,-0.423812,-0.571957,-0.248678,-1.184498,-0.218058,0.481581,-0.324890,-0.557771,0.368504,-0.252583,-0.638496,0.041190,-0.745992,-0.032350,0.438188,-0.233441,0.052832,0.392574,-0.152052,0.265614,-0.452502,0.047997,-0.527529,-0.453233,-0.096846,0.457156,-0.555910,0.878715,-0.401071,-1.019632,0.864250,-0.332461,-0.536304,-0.196656,-0.126987,-0.187574,0.308931,0.0,-0.231580,0.389358,0.394364,-0.645213,-1.090741,0.544154,-0.539020,-0.409401,0,0,0,0,0,0,0,1
3,4084,4,-0.145749,0.127016,0.887275,-0.423812,-0.490685,-0.248678,-1.206385,1.502279,1.110824,-0.495978,-0.477353,0.250149,-0.007558,-0.624446,-0.258374,-0.745992,-0.268389,0.409374,1.481021,0.044427,0.283235,-0.263852,0.129467,-0.762581,-0.250632,-0.700651,-0.453233,0.135097,0.240202,-0.775073,0.860388,-0.436922,-0.936779,0.847388,-0.332461,-0.783592,-0.196656,-0.126987,-0.187574,1.452899,0.0,-0.231580,-0.419912,0.284934,-0.645213,-1.113821,1.277342,-0.339869,-0.488065,0,0,0,0,0,0,0,1
4,4084,5,-0.145749,0.264979,-0.677674,0.235579,-0.762838,-0.248678,-1.372699,0.330435,0.221893,-0.088335,-0.775517,0.369856,0.736467,-0.013313,-0.258374,-0.116431,-0.032350,-0.825734,0.363659,0.066077,0.284984,0.127448,0.267170,-0.685061,0.047997,-0.634066,-0.453233,0.634827,0.240202,-0.702019,0.841450,-0.392108,0.978465,0.830527,0.580794,-0.804318,-0.196656,-0.126987,-0.187574,0.829583,0.0,-0.231511,0.389358,0.286685,-0.304185,-1.111499,0.195017,-0.726550,-0.330738,0,0,0,0,0,0,0,1


In [51]:
normalized_test_data["score"] = best_model.predict(normalized_test_data[informative_features])
result = normalized_test_data.groupby(["id"]).mean()["score"].round().astype(int)
result = pd.DataFrame(result)
result.head()

,score
id,
4084,0
4085,0
4086,0
4087,0
4088,0


In [52]:
result.to_csv("output.csv", header=result.columns)